In [ ]:
import cv2
import numpy as np
import string
import datetime
import collections
import json
import sys
import math

In [ ]:
def shutdown(cam):
    cv2.destroyAllWindows()
    cam.release()

In [ ]:
def extractRect(img, x, y, w, h):
    if img is None:
        return None
    
    return img[y:y+h,x:x+w]

In [ ]:
def removeBackground(img):
    img[np.where((img>=[220,220,220]).all(axis=2))] = [0,0,0]

In [ ]:
def drawCircle(img, centre, radius, colour, thickness):
    cv2.circle(img, (centre[0],centre[1]), radius, (colour,colour,colour), thickness)

In [ ]:
def maskCentre(img, centre, radius):
    drawCircle(img, centre, radius, 0, -1)

In [ ]:
def maskOutside(img, centre, radius):
    thickness = 1000
    drawCircle(img, centre, int(radius + (thickness / 2)), 0, thickness)

In [ ]:
def capture(cam):
    raw = cam.read()[1]
    
    if raw is None:
        return None
    
    removeBackground(raw)
    maskCentre(raw, centrePoint, centreRadius)
    maskOutside(raw, centrePoint, outsideRadius)
    
    gray = cv2.cvtColor(raw, cv2.COLOR_RGB2GRAY)
            
    return gray

In [ ]:
def drawRect(img, x, y, w, h, colour):
    cv2.rectangle(img,(x,y),(x+w,y+h),(colour,colour,colour), 1)

In [ ]:
def threshold(img):
    retval, threshold = cv2.threshold(img, 150, 255, cv2.THRESH_OTSU)
    
    return threshold

In [ ]:
def allEqual(aList):
    return all(aList[0] == item for item in aList)

In [ ]:
class Zone:
    
    def __init__(self, x, y, size):
        self.__x = x
        self.__y = y
        self.__w = size
        self.__h = size
        self.__hot = None
    
    def x(self):
        return self.__x
    
    def y(self):
        return self.__y
    
    def w(self):
        return self.__w
    
    def h(self):
        return self.__h
    
    def isHot(self):
        return self.__hot
    
    def update(self, data):
        zoneRect = extractRect(data, self.x(), self.y(), self.w(), self.h())
        self.__hot = isLighterThan50percentGrey(zoneRect)

In [ ]:
def isLighterThan50percentGrey(blackAndWhiteImg):
    return cv2.mean(blackAndWhiteImg)[0] < 128.0

In [ ]:
class Viewer:
    
    def show(self, frame):
        key = cv2.waitKey(10)
        cv2.imshow('frame', frame)
        
    def render(self, rawFrame, filteredFrame, meter):
        key = cv2.waitKey(10)
        
        rawCopy = rawFrame.copy()
        filterCopy = filteredFrame.copy()
        
        for zone in meter.getZones():
            
            zoneImg = extractRect(filterCopy, zone.x(), zone.y(), zone.w(), zone.h())
            
            drawRect(rawCopy, zone.x(), zone.y(), zone.w(), zone.h(), 0)
            if zone.isHot():
                drawRect(filterCopy, zone.x(), zone.y(), zone.w(), zone.h(), 255)
            else:
                drawRect(filterCopy, zone.x(), zone.y(), zone.w(), zone.h(), 0)
                
        cv2.imshow('raw', rawCopy)
        cv2.imshow('filtered', filterCopy)

class NullViewer:
    def show(self, frame):
        return
    
    def render(self, rawFrame, filteredFrame, meter):
        return

In [ ]:
class Monitor:
    
    def __init__(self, camera, meter, viewer = NullViewer()):
        self.__camera = camera
        self.__meter = meter
        self.__online = True
        self.__viewer = viewer
    
    def poll(self):
        newFrame = capture(self.__camera)
        self.__online = newFrame is not None
        
        if self.__online:
            filteredFrame = self.filterFrame(newFrame)
            flowQty = self.__meter.update(filteredFrame)
            
            self.__viewer.render(newFrame, filteredFrame, self.__meter)
            
            return flowQty
        else:
            if not calibrate:
                raise Exception("camera offline!")
    
    def isOnline(self):
        return self.__online
    
    def filterFrame(self, rawFrame):
        return threshold(rawFrame)
    

In [ ]:
class Trigger:
    
    def __init__(self, zone1, zone2):
        self.__zone1 = zone1
        self.__zone2 = zone2
        self.__lastState = [None, None]
        self.__state = [None, None]
        
        self.__validStates = collections.deque(maxlen=4)
        self.__validStates.append([True,True])
        self.__validStates.append([False,True])
        self.__validStates.append([False,False])
        self.__validStates.append([True,False])
    
    def setNumber(self, num):
        self.__num = num
        
    def zones(self):
        return [self.__zone1, self.__zone2]
        
    def update(self, data):
        self.__zone1.update(data)
        self.__zone2.update(data)
        
        self.__lastState = list(self.__state)
        self.__state = [self.__zone1.isHot(), self.__zone2.isHot()]
        
        if not self.__knownState():
            while self.__validStates[0] != self.__state:
                self.__validStates.rotate(-1)
    
    def fired(self):
        if self.__hasChanged() and self.__knownState():     
            
            self.__validStates.rotate(-1)
            if self.__validStates[0] == self.__state:
                if allEqual(self.__state):
                    if debug:
                        print(self.__num, " : ", self.__lastState, " -> ", self.__state)
                    return True
            else:
                if not calibrate:
                    raise Exception('error on trigger', self.__num)
        return False
            
    
    def __hasChanged(self):
        return set(self.__lastState) != set(self.__state)
    
    def __knownState(self):
        return None not in self.__lastState
    
  

In [ ]:
class Meter:
    
    def __init__(self, name, triggers, sensitivity):
        self.__triggers = triggers
        self.__zones = []
        self.__name = name
        self.__lastFired = None
        self.__sensitivity = sensitivity
        self.__fireDeque = collections.deque(maxlen=len(triggers))
        
        trigCount = 0
        for item in triggers:
            self.__fireDeque.append(item)
            self.__zones.extend(item.zones())
            item.setNumber(trigCount)
            trigCount = trigCount + 1
        
    def update(self, data):
        fired = []
        for trigger in self.__triggers:
            trigger.update(data)
            if trigger.fired():
                fired.append(trigger)
            
        if len(fired) > 1 and not calibrate:
            raise Exception("Two triggers fired together?")
        
        if len(fired) == 1:
            if self.__lastFired is None:
                while self.__fireDeque[0] is not fired[0]:
                    self.__fireDeque.rotate(-1)
                self.__fireDeque.rotate(1)
            
            self.__lastFired = fired[0]
            self.__fireDeque.rotate(-1)
            
            if self.__fireDeque[0] is not self.__lastFired and not calibrate:
                raise Exception("Unexpected trigger fired!")
            else:
                return self.__sensitivity
        
        return 0
            
        
    def getZones(self):
        return self.__zones

In [ ]:
def arrayToZone(array):
    x = array[0]
    y = array[1]
    size = array[2]
    
    left = int(x-(size/2))
    top = int(y-(size/2))
    
    return Zone(left, top, array[2])

In [ ]:
def warmUp(cam, viewer, frames):
    count = 0
    while(count < frames):
        img = capture(cam)
        viewer.show(img)
        count = count+1


In [ ]:
def rotate_around_point_lowperf(point, radians, origin=(0, 0)):
    """
    From https://ls3.io/post/rotate_a_2d_coordinate_around_a_point_in_python/
    """
    x, y = point
    ox, oy = origin

    qx = ox + math.cos(radians) * (x - ox) + math.sin(radians) * (y - oy)
    qy = oy + -math.sin(radians) * (x - ox) + math.cos(radians) * (y - oy)

    return qx, qy

In [ ]:
def degreesToClockwiseRads(degrees):
    return math.radians(degrees) * -1

In [ ]:
def getZoneByAngle(degrees, radiusOffset, size):    
    zeroAnglePoint = (origin[0] - triggerRadius - radiusOffset, origin[1])
    zeroAnglePoint = rotate_around_point_lowperf(zeroAnglePoint, degreesToClockwiseRads(zeroAngle), origin)
    rotatedPoint = rotate_around_point_lowperf(zeroAnglePoint, degreesToClockwiseRads(degrees), origin)
    rotatedAngleZone = arrayToZone([rotatedPoint[0],rotatedPoint[1],size])
    
    return rotatedAngleZone

In [ ]:
def getZoneFromConfig(item):
    return getZoneByAngle(item["angle"], item["offset"], item["size"])

In [ ]:
config = json.load(open('config.json'))
src = config["captureSource"]
mode = config["mode"]
calibrate = "CALIBRATE" in mode
centrePoint = config["meterFace"]["centrePoint"]
centreRadius = config["meterFace"]["radius"]["inner"]
outsideRadius = config["meterFace"]["radius"]["outer"]
triggerRadius = config["meterFace"]["radius"]["trigger"]
zeroAngle = config["meterFace"]["zeroAngle"]

debug = "DEBUG" in mode

triggers = []
 
origin = (centrePoint[0], centrePoint[1])

for trigger in config["triggers"]:
    zone0 = getZoneFromConfig(trigger[0])
    zone1 = getZoneFromConfig(trigger[1])
    triggers.append(Trigger(zone0,zone1))
    

meter = Meter("hot", triggers, config["sensitivity"])

viewer = NullViewer()
if calibrate:
    viewer = Viewer()

In [ ]:
try:
    cam = cv2.VideoCapture(src)
    warmUp(cam, NullViewer(), 30)

    monitor = Monitor(cam, meter, viewer)
    while(monitor.isOnline()):
        flowQty = monitor.poll()

        if flowQty is not None and flowQty>0:
            if debug:
                print(flowQty)
#         todo - send readings somewhere
except:
    print("Unexpected error:", sys.exc_info())
    shutdown(cam)